
<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-11t1vbu4x-xkBIHmOREQnYnYDH1GDfCg">Community</a>
    </p>
</center>

# <center>Tracing with CrewAI</center>    

This guide demonstrates how to use Arize with CrewAI for tracing. 

You can read more about LLM tracing [here](https://docs.arize.com/arize/llm-large-language-models/llm-traces). In this tutorial, you will use opentelemetry and [openinference](https://github.com/Arize-ai/openinference/tree/main) to instrument our application in order to trace your LLM and agent calls with CrewAI.

ℹ️ This notebook requires:
- A OpenAI API key
- A Serper API key (this is at [serper.dev](https://serper.dev) and has a free tier)
- An Arize Space & API Key


## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies.

In [3]:
# Dependencies needed to instrument your openai application using opentelemetry and openinference
!pip -q install "arize-otel>=0.7.0" "openinference-instrumentation-crewai>=0.1.3" "openinference-instrumentation-litellm>=0.1.5"

# External dependencies needed to use crewai, export spans and send them to our collectors: Arize
!pip install -q opentelemetry-sdk==1.28.0 opentelemetry-exporter-otlp==1.28.0 opentelemetry-instrumentation-httpx crewai==0.86.0 crewai-tools==0.17.0

## Step 2: Get your API Keys

Copy the Arize API_KEY and SPACE_ID from your Space Settings page (shown below) to the variables in the cell below.

<center><img src="https://storage.googleapis.com/arize-assets/barcelos/Screenshot%202024-11-11%20at%209.28.27%E2%80%AFPM.png" width="700"></center>

In [3]:
import os
from getpass import getpass

if not (SPACE_ID := os.getenv("SPACE_ID")):
    SPACE_ID = getpass("🔑 Enter your Space ID: ")

if not (ARIZE_API_KEY := os.getenv("ARIZE_API_KEY")):
    ARIZE_API_KEY = getpass("🔑 Enter your Arize API Key: ")

if not (OPENAI_API_KEY := os.getenv("OPENAI_API_KEY")):
    OPENAI_API_KEY = getpass("🔑 Enter your OpenAI API key: ")

if not (SERPER_API_KEY := os.getenv("SERPER_API_KEY")):
    SERPER_API_KEY = getpass("🔑 Enter your Serper API key: ")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

## Step 3. Add our tracing code
We will be using the `arize-otel` package to register the URL and authentication parameters to send to Arize using OpenTelemetry. You can see what's under the hood by looking [here](https://docs.arize.com/arize/large-language-models/tracing/auto-instrumentation).

In [4]:
# Import open-telemetry dependencies
from arize.otel import register
from openinference.instrumentation.crewai import CrewAIInstrumentor
from openinference.instrumentation.litellm import LiteLLMInstrumentor

# Setup OTEL via our convenience function
tracer_provider = register(
    space_id=SPACE_ID,  # in app space settings page
    api_key=ARIZE_API_KEY,  # in app space settings page
    project_name="crewai-tracing-tutorial",  # name this to whatever you would like
)

CrewAIInstrumentor().instrument(tracer_provider=tracer_provider)
LiteLLMInstrumentor().instrument(tracer_provider=tracer_provider)

## Step 4: Run your LLM application
Let's create two agents that work together to write a blog post and see the results in Arize.

In [5]:
from crewai import Agent, Crew, Process, Task
from crewai_tools import SerperDevTool

search_tool = SerperDevTool()

# Define your agents with roles and goals
researcher = Agent(
    role="Senior Research Analyst",
    goal="Uncover cutting-edge developments in AI and data science",
    backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
    verbose=True,
    allow_delegation=False,
    # You can pass an optional llm attribute specifying what model you wanna use.
    # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7),
    tools=[search_tool],
)
writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True,
    allow_delegation=True,
)

# Create tasks for your agents
task1 = Task(
    description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer,
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=1,
    process=Process.sequential,
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

2024-12-05 16:50:32,306 - 8342091840 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Senior Research Analyst
## Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.


# Agent: Senior Research Analyst
## Thought: I need to conduct a comprehensive analysis of the latest advancements in AI for 2024. This requires me to gather information about key trends, breakthrough technologies, and potential industry impacts. I will perform an internet search to find relevant and up-to-date information.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"latest advancements in AI 2024 key trends breakthrough technologies industry impacts\"}"
## Tool Output: 

Search results: Title: 11 New Technologies in AI: All Trends of 2023-2024 - devabit
Link: https://devabit.com/blog/top-11-new-technologies-in-ai-exploring-the-latest-trends/
Snippet: From the remarkable advancements in natural language processing (NLP) and GPT to the impact of AI on the sales

2024-12-05 16:50:42,761 - 8342091840 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types




# Agent: Senior Research Analyst
## Final Answer: 
- **Key Trends in AI for 2024:**
  - **Generative AI Mainstreaming**: Rapid adoption of generative AI across various industries, enabling creative content generation, personalized marketing, and product design.
  - **Multimodal AI**: Increased focus on AI systems that can process and integrate multiple forms of input (text, audio, video) for enhanced understanding and interaction.
  - **AI Democratization**: Expanding access to AI tools for businesses of all sizes, making advanced technology available to small and medium enterprises (SMEs).
  - **Scaling Challenges**: Companies face difficulties in scaling AI solutions due to complex infrastructure requirements, ongoing training, and talent shortages.
  - **Sustainable AI Practices**: Growing emphasis on employing AI in ways that address climate change and resource conservation, such as optimizing energy consumption.

- **Breakthrough Technologies:**
  - **Agentic AI**: Development o

2024-12-05 16:50:54,691 - 6251540480 - __init__.py-__init__:164 - WARNING: Transient error Bad Gateway encountered while exporting span batch, retrying in 1s.


2024-12-05 16:51:12,001 - 8342091840 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types




# Agent: Tech Content Strategist
## Final Answer: 
**The Future is Now: Key AI Advancements Shaping 2024**

As we dive headfirst into 2024, the landscape of artificial intelligence (AI) is more electrifying than ever. With innovations unfolding at breakneck speed, AI is not just a buzzword anymore—it's a transformative force in industries across the globe. Let's unpack some of the most significant advancements that are taking center stage this year.

One of the standout trends is the mainstreaming of *Generative AI*. This technology is revolutionizing how businesses create content, from personalized marketing materials to sophisticated product designs. Companies are harnessing tools that can generate high-quality text, images, and even music, allowing for a level of creativity previously thought impossible. Imagine a marketing team crafting tailored campaigns that resonate deeply with specific audiences—that's the power of generative AI in action. It’s rapidly becoming essential for 

Great! Our application works!

## Step 5: Log into Arize and explore your application traces 🚀

Log into your Arize account, and look for the project with the same `project_name`!